In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from load_data import load_data, split_data

data = load_data("1H")
all_columns = data.keys()
all_columns

Index(['MWh', 'solar_fore_de [MW]', 'solar_fore_it [MW]', 'wind_fore_de [MW]',
       'wind_fore_it [MW]', 'CH_AT', 'CH_DE', 'CH_FR', 'CH_IT', 'AT_CH',
       'DE_CH', 'FR_CH', 'IT_CH'],
      dtype='object')

In [2]:
columns_to_drop = [
    "CH_AT",
    "CH_DE",
    "CH_FR",
    "CH_IT",
    "AT_CH",
    "DE_CH",
    "FR_CH",
    "IT_CH",
]

In [3]:
data_filtered = data.drop(columns=columns_to_drop)
data_filtered

,MWh,solar_fore_de [MW],solar_fore_it [MW],wind_fore_de [MW],wind_fore_it [MW]
2019-01-01 00:00:00+00:00,129.716036,0.0,0.0,23052.3310,4596.5916
2019-01-01 01:00:00+00:00,133.398074,0.0,0.0,24969.9701,4478.5564
2019-01-01 02:00:00+00:00,135.133852,0.0,0.0,27082.9626,4323.3712
2019-01-01 03:00:00+00:00,131.699424,0.0,0.0,26890.9717,4231.8283
2019-01-01 04:00:00+00:00,147.391128,0.0,0.0,27740.1555,4266.3082
...,...,...,...,...,...
2021-12-31 18:00:00+00:00,171.707318,0.0,0.0,36997.7200,1108.4000
2021-12-31 19:00:00+00:00,159.462903,0.0,0.0,35666.9300,1077.9700
2021-12-31 20:00:00+00:00,155.109520,0.0,0.0,34383.8800,1048.2800
2021-12-31 21:00:00+00:00,171.370277,0.0,0.0,33075.2500,1078.7800


In [4]:
sep = len(data_filtered) // 3 * 2

train_df = data_filtered[: sep]
test_df = data_filtered[sep :]

X_train, metadata_train, y_train = split_data(train_df, 7 * 24, 24, 1, "MWh")
X_test, metadata_test, y_test = split_data(test_df, 7 * 24, 24, 1, "MWh")

X_train.shape, metadata_train.shape, y_train.shape, X_test.shape

((17343, 168), (17343, 5, 24), (17343, 24), (8578, 168))

In [ ]:
import lightgbm as lgb

params = {
    'n_estimators': 2000,
    'max_depth': 4,
    'num_leaves': 2**4,
    'learning_rate': 0.1,
    #'boosting_type': 'dart',
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "mae",
    "random_state": 42,
    "device": "gpu",
}

model = lgb.LGBMRegressor(first_metric_only = True, **params)

model.fit(X_train, y_train[:, 0],
          eval_metric = 'l1', 
          eval_set = [(X_test, y_test[:, 0])])

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 42840
[LightGBM] [Info] Number of data points in the train set: 17343, number of used features: 168
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 168 dense feature groups (2.78 MB) transferred to GPU in 0.037363 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 106.790242
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

In [ ]:
import plotly.express as px
plot_df = {"epochs": epochs, "Train MAE": train_mae_list, "Test MAE": test_mae_list}
px.line(plot_df, x="epochs", y=["Train MAE", "Test MAE"])

In [ ]:
train_mae_list[0].item()